# Clifford randomized benchmarking

In [ ]:
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

import numpy as np
from qutip import sigmax, sigmay, sigmaz, basis, qeye, tensor, Qobj, fock_dm
from qutip_qip.circuit import QubitCircuit, Gate, CircuitSimulator
from qutip_qip.device import ModelProcessor, Processor, Model
from qutip_qip.compiler import GateCompiler, Instruction
from qutip_qip.operations import gate_sequence_product

## Create pulse compiler
 

In [ ]:
class MyModel(Model):
    """A custom Hamiltonian model with sigmax and sigmay control."""
    def get_control(self, label):
        """
        Get an available control Hamiltonian.
        For instance, sigmax control on the zeroth qubits is labeled "sx0".

        Args:
            label (str): The label of the Hamiltonian

        Returns:
            The Hamiltonian and target qubits as a tuple (qutip.Qobj, list).
        """
        targets = int(label[2:])
        if label[:2] == "sx":
            return 2 * np.pi * sigmax() / 2, [targets]
        elif label[:2] == "sy":
            return 2 * np.pi * sigmay() / 2, [targets]
        else:
            raise NotImplementError("Unknown control.")


class MyCompiler(GateCompiler):
    """Custom compiler for generating pulses from gates using the base class 
    GateCompiler.

    Args:
        num_qubits (int): The number of qubits in the processor
        params (dict): A dictionary of parameters for gate pulses such as
                       the pulse amplitude.
    """

    def __init__(self, num_qubits, params=None, args=None):
        super().__init__(num_qubits, params=params)
        self.params = params
        self.gate_compiler = {
            "RX": self.single_qubit_gate_compiler,
            "RY": self.single_qubit_gate_compiler,
        }
        self.args = {  # Default configuration
            "shape": "rectangular",
            "num_samples": None,
            "params": self.params,
        }
        self.args.update( args )
        print(args)

    def generate_pulse(self, gate, tlist, coeff, phase=0.0):
        """Generates the pulses.

        Args:
            gate (qutip_qip.circuit.Gate): A qutip Gate object.
            tlist (array): A list of times for the evolution.
            coeff (array): An array of coefficients for the gate pulses
            phase (float): The value of the phase for the gate.

        Returns:
            Instruction (qutip_qip.compiler.instruction.Instruction): An instruction
            to implement a gate containing the control pulses.                                               
        """

        pulse_info = [
            # (control label, coeff)
            ("sx" + str(gate.targets[0]), np.cos(phase) * coeff),
            ("sy" + str(gate.targets[0]), np.sin(phase) * coeff),
        ]
        #print(tlist)
        return [Instruction(gate, tlist=tlist, pulse_info=pulse_info)]

    def single_qubit_gate_compiler(self, gate, args):
        """Compiles single-qubit gates to pulses.
        
        Args:
            gate (qutip_qip.circuit.Gate): A qutip Gate object.
        
        Returns:
            Instruction (qutip_qip.compiler.instruction.Instruction): An instruction
            to implement a gate containing the control pulses.
        """
        
        
        # gate.arg_value is the rotation angle
        coeff, tlist= self.generate_pulse_shape(
                "hann",  # Scipy Hann window
                self.args["num_samples"],  # 100 sampling point
                maximum= 1,
                area= 1,  # 1/2 becuase we use sigmax as the operator instead of sigmax/2
            )
        # tlist = np.abs(gate.arg_value) / self.params["pulse_amplitude"]
        coeff *= gate.arg_value/np.pi
        if gate.name == "RX":
            return self.generate_pulse(gate, tlist, coeff, phase=0.0)
        elif gate.name == "RY":
            return self.generate_pulse(gate, tlist, coeff, phase=np.pi / 2)

Create Cilfford group gates

In [ ]:
## Test
"""
g_x = Gate("X", 0)
g_y = Gate("Y", 0)
g_z = Gate("Z", 0)
rg_z = Gate("RZ", 0, arg_value= np.pi)
g_nhz = Gate("RZ", 0, arg_value= np.pi/2)
"""


## Basic
## Pi
rg_i = Gate("RX", 0, arg_value= 0)
rg_x = Gate("RX", 0, arg_value= np.pi)
rg_y = Gate("RY", 0, arg_value= np.pi)
## Pi/2
rg_px2 = Gate("RX", 0, arg_value= +np.pi/2)
rg_py2 = Gate("RY", 0, arg_value= +np.pi/2)
rg_nx2 = Gate("RX", 0, arg_value= -np.pi/2)
rg_ny2 = Gate("RY", 0, arg_value= -np.pi/2)

## Decompose
## Pi
g_z = [rg_y,rg_x]
## Pi/2
g_phz = [rg_nx2,rg_py2,rg_px2]
g_nhz = [rg_nx2,rg_ny2,rg_px2]
## Had
g_hpxz = [rg_x,rg_ny2]
g_hnxz = [rg_x,rg_py2]
g_hpyz = [rg_y,rg_px2]
g_hnyz = [rg_y,rg_nx2]
g_hpxy = [rg_px2,rg_py2,rg_px2]
g_hnxy = [rg_nx2,rg_py2,rg_nx2]
## 2pi/3 
g_pc1 = [rg_py2,rg_px2]
g_pc2 = [rg_py2,rg_nx2]
g_pc4 = [rg_ny2,rg_px2]
g_pc3 = [rg_ny2,rg_nx2]

g_nc1 = [rg_nx2,rg_py2]
g_nc2 = [rg_px2,rg_py2]
g_nc4 = [rg_nx2,rg_ny2]
g_nc3 = [rg_px2,rg_ny2]






In [ ]:
gates_set = [
    [rg_i],[rg_x],[rg_y],[rg_px2],[rg_nx2],[rg_py2],[rg_ny2],
    ## Pi
    g_z,
    ## Pi/2
    g_phz,g_nhz,
    ## Had
    g_hpxz,g_hnxz,g_hpyz,g_hnyz,g_hpxy,g_hnxy,
    ## 2pi/3 
    g_pc1,g_pc2,g_pc4,g_pc3,
    g_nc1,g_nc2,g_nc4,g_nc3
]

gate_set_num = len(gates_set)




In [ ]:
def decomposition( gates ):
    circuit = QubitCircuit(1)
    eff_op = qeye(2)
    name_seq = []
    for g in gates:
        circuit.add_gate(g)
        name_seq.append(g.name)
        g_qobj = g.get_compact_qobj()
        eff_op = g_qobj *eff_op
        # eff_op = circuit.run(qeye(2))
    #print(name_seq)
    return eff_op

In [ ]:
success_i = 0
for i in range(10):
    
    circuit = QubitCircuit(1)
    single_qubit = basis(2)
    total_op = qeye(2)
    num_gates = 5
    for ind in np.random.randint(0, gate_set_num, num_gates):
        random_gate = gates_set[ind]
        eff_op = decomposition(random_gate)
        for g in random_gate:
            circuit.add_gate(g)
        #print(random_gate.name, random_gate.arg_value/np.pi)
        #print( eff_op )
        single_qubit = eff_op*single_qubit
        total_op = eff_op*total_op
            #print( total_op )

    #print(total_op*total_op.inv())

    # Test all operation
    for gate in gates_set:
        name_seq = []
        rev_op = total_op.inv()
        compared_op = decomposition(gate)
        #print(compared_op*total_op, rev_op*total_op)
        final_qubit = compared_op*single_qubit
        #print( final_qubit[0][0][0]**2 )
        for g_phase in [1,1j,-1,-1j]:
            
            if g_phase*rev_op == compared_op:
                #print(g_phase*total_op, eff_re_op, eff_re_op*g_phase*total_op)
                #print( g_phase, "ground state", abs(final_qubit[0][0][0]))
                
                print( abs(final_qubit[0][0][0]), "ground state")
                success_i+=1
                
print(success_i)

In [ ]:
# Simulate the circuit.
myprocessor = ModelProcessor(model=MyModel(1))
myprocessor.native_gates = ["RX","RY"]
mycompiler = MyCompiler(1, args={"num_samples": 20})
#print(mycompiler.gate_compiler)

# raw circuit
# for gate in circuit.gates:
#     print(gate.name, gate.get_compact_qobj())

tlist, coeffs = myprocessor.load_circuit(circuit, compiler=mycompiler)

# print("tlist",tlist)
# print("coeffs",coeffs)

plt.plot(tlist["sx0"],coeffs["sx0"])
plt.show()
# result = myprocessor.run_state(basis(2, 0))

# fig, ax = myprocessor.plot_pulses(
#     figsize=(5, 3), dpi=120,
#     use_control_latex=False
# )
# ax[-1].set_xlabel("$t$")
# fig.tight_layout()